# Using RAGAs to Evaluate Model

## _Why evaluate a RAG_?

Evaluation tells you what actually works and what fails. For a RAG system you’re not just testing an LLM — you’re testing a pipeline:

Retriever quality (are relevant chunks returned?)

Generator quality given context (is the answer correct/faithful?)

End-to-end user experience (latency, cost, robustness, safety)

Without evaluation you’ll be fixing the wrong thing (tuning prompts when retrieval is the real problem, or vice versa).

- [x] create a testset
- [ ] what all is needed in the test set? 
- [ ] why do we need the test set?
- [ ] 

### Getting Transcript Data of the video

In [28]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [29]:
ytt_api = YouTubeTranscriptApi()
video_id = "P14cRV-m6ZY"
fetched_transcript = ytt_api.fetch(video_id,languages=["en","hi"])

transcript = " ".join(snippet.text for snippet in fetched_transcript)

In [30]:
transcript

"You know, whenever I read the Bhagavad Gita, especially commentaries upon it, explanations, there are different approaches to it. Some say the central teaching of the Gita is bhakti, devotion. Some say the central teaching of the Gita is action. Sri Krishna relentlessly urges Arjuna into action. Some say it is duty. Another way of looking at the Bhagavad Gita is that it is a manual on spiritual life with teachings about God and avatar and meditation and so on. All of which, no doubt, are true. But if you actually look at the Bhagavad Gita itself, when Sri Krishna starts teaching Arjuna, the first thing he teaches Arjuna is probably the central message of Vedanta. I'm saying it very carefully, central message of Vedanta. I did not say central message of Gita because the central message of Gita and the central message of Vedanta are one and the same. The Gita is part of Vedanta. Gita in itself may have a unique approach to it. There may be something special in the Gita over and above th

In [31]:
def get_transcript(video_id):
   ytt_api = YouTubeTranscriptApi()
   fetched_transcript = ytt_api.fetch(video_id,languages=["en","hi"])

   transcript = " ".join(snippet.text for snippet in fetched_transcript)
   return transcript

In [32]:
from langchain_community.document_loaders import TextLoader

transcript_file = "./transcript.txt"
loader = TextLoader(transcript_file)
documents = loader.load()

# so we basically need a `Document` object to proceed either create the object from the text you have or provide the file. i've choosen the latter way

In [33]:
documents

[Document(metadata={'source': './transcript.txt'}, page_content="You know, whenever I read the Bhagavad Gita, especially commentaries upon it, explanations, there are different approaches to it. Some say the central teaching of the Gita is bhakti, devotion. Some say the central teaching of the Gita is action. Sri Krishna relentlessly urges Arjuna into action. Some say it is duty. Another way of looking at the Bhagavad Gita is that it is a manual on spiritual life with teachings about God and avatar and meditation and so on. All of which, no doubt, are true. But if you actually look at the Bhagavad Gita itself, when Sri Krishna starts teaching Arjuna, the first thing he teaches Arjuna is probably the central message of Vedanta. I'm saying it very carefully, central message of Vedanta. I did not say central message of Gita because the central message of Gita and the central message of Vedanta are one and the same. The Gita is part of Vedanta. Gita in itself may have a unique approach to 

In [34]:
config = {
    "model": "llama-3.3-70b-versatile",  # or other model IDs
    "temperature": 0.7,
    "max_tokens": None,
    "top_p": 0.8,
}

In [35]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
from langchain_groq import ChatGroq
from ragas.embeddings import OpenAIEmbeddings
import os

# llm model is groq 
generator_llm = LangchainLLMWrapper(ChatGroq(
   model=config["model"],
   temperature=config['temperature']
))

# embedding model is google embedding model 
generator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"))

/var/folders/2b/js51tfts6z70ngzhlwh5zfvh0000gn/T/ipykernel_52883/919848536.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  generator_llm = LangchainLLMWrapper(ChatGroq(
E0000 00:00:1759887127.760853 1875341 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
/var/folders/2b/js51tfts6z70ngzhlwh5zfvh0000gn/T/ipykernel_52883/919848536.py:13: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_

In [37]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 